In [1]:
# Author: Selma Gomez Orr <selmagomezorrds@gmail.com> January 30, 2016

##########################################################################
## Imports
##########################################################################

import requests, urllib2, gzip, urllib
import pandas as pd
import numpy as np
import json
import ujson
import dill


import numpy as np

##########################################################################
## Constants
##########################################################################

url = "http://thedataincubator.s3.amazonaws.com/coursedata/mldata/yelp_train_academic_dataset_review.json.gz"


##########################################################################
## Program - Miniproject: nlp Bag of Words
##########################################################################


#Download file
urllib.urlretrieve(url,'Reviews')

reviews_list = []
with gzip.open('Reviews', 'rb') as f:
    for line in f.readlines():
        review = ujson.loads(line)
        reviews_list.append(review)
        
print reviews_list[0:2]





[{u'votes': {u'funny': 0, u'useful': 2, u'cool': 1}, u'user_id': u'Xqd0DzHaiyRqVH3WRG7hzg', u'review_id': u'15SdjuK7DmYqUAj6rjGowg', u'text': u"dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.", u'business_id': u'vcNAWiLM4dR7D2nwwJ7nCA', u'stars': 5, u'date': u'2007-05-17', u'type': u'review'}, {u'votes': {u'funny': 0, u'useful': 2, u'cool': 0}, u'user_id': u'H1kH6QZV7Le4zqTRNxoZow', u'review_id': u'RF6UnRTtG7tWMcrO2GEoAg', u'text': u"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat 

In [2]:
#Cereate a dataframe with the text and star ratings.
fields = ['text', 'stars']

df_reviews = pd.DataFrame(reviews_list, columns = fields)

print df_reviews.head()
df_reviews.describe()


                                                text  stars
0  dr. goldberg offers everything i look for in a...      5
1  Unfortunately, the frustration of being Dr. Go...      2
2  Dr. Goldberg has been my doctor for years and ...      4
3  Been going to Dr. Goldberg for over 10 years. ...      4
4  Got a letter in the mail last week that said D...      4


,stars
count,1012913.000000
mean,3.737691
std,1.299111
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [5]:
#Imports from Scikit Learn
from sklearn.cross_validation import train_test_split as tts
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
#Get data and target from dataframe and convert to format for sklearn.
target = np.array(df_reviews['stars'])
target = np.ravel(target)
data = np.array(df_reviews['text'])

splits = tts(data, target, test_size=0.2)
X_train, X_test, y_train, y_test = splits


text_clf = Pipeline([
        ('hash', HashingVectorizer(ngram_range= (1,2))),
        ('clf', linear_model.Ridge())
    ])
        
parameters = {'clf__alpha' : (.055, .05, 1)}  

gs_text_clf = GridSearchCV(text_clf, parameters, n_jobs = -1)

gs_text_clf = gs_text_clf.fit(X_train[:10000], y_train[:10000])

print gs_text_clf.best_params_

print gs_text_clf.score(X_test, y_test)



    

{'clf__alpha': 0.055}
0.562220592193


In [ ]:
big_clf = Pipeline([
        ('hash', HashingVectorizer(ngram_range = (1,2))),
        ('clf', linear_model.Ridge(alpha=1))
    ])

big_clf = big_clf.fit(data,target)





In [ ]:


#Store the instance using dill.
with open('big_file','w') as f:
    dill.dump(big_clf,f)
    
    
    

In [ ]:
import dill

big_clf = dill.load(open('big_file', 'r'))

answer = big_clf.predict(["Love it!!!!! Love it!!!!!! love it!!!!!!!   Who doesn't love Culver's!"])

print answer
